In [39]:
import __init__
import pickle
import numpy as np 
import pandas as pd 
from data_set import EnsembleModelReporter
from sklearn import cross_validation
from sklearn.metrics import accuracy_score

In [40]:
%time data = pickle.load(open('training_set_3', 'r+'))
reporter = pickle.load(open('ensemble_model_reporter', 'r+'))

CPU times: user 2.12 s, sys: 757 ms, total: 2.88 s
Wall time: 2.93 s


In [41]:
test_feature = pickle.load(open('testing_set_feature', 'r+'))

In [65]:
test_feature.rename(columns={'year_counter': 'date_sign'},inplace=True)

In [42]:
reporter.get_mean_loss(threshold=1)

0.09075177145156807

In [43]:
shop_ids = range(1, 2001)

In [44]:
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso, BayesianRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor

models = {
    'xgboost': XGBRegressor(max_depth=10, learning_rate=0.1, n_estimators=30, nthread=-1, gamma=0, min_child_weight=1),
    'lasso': Lasso(alpha=1),
    'bayesian_ridge': BayesianRidge(),
    'svr': SVR(),
    'kneighbors': KNeighborsRegressor(),
    'decision_tree': DecisionTreeRegressor(),
    'ramdom_forest': RandomForestRegressor(n_estimators=10, n_jobs=-1),
    'adaboost': AdaBoostRegressor(),
    'bagging': BaggingRegressor(n_jobs=-1),
    'gbr_regressor': GradientBoostingRegressor(n_estimators=500, max_depth=4, 
                                               min_samples_split=2, learning_rate=0.01, loss='ls')
}

In [59]:
data.columns

Index([u'shop_id', u'city_name', u'location_id', u'per_pay', u'score',
       u'comment_cnt', u'shop_level', u'cate_1_name', u'cate_2_name',
       u'cate_3_name', u'date', u'high_temp', u'low_temp', u'rainy',
       u'wind_force', u'date_sign', u'month_counter', u'day_counter',
       u'holiday', u'businessday', u'weekday', u'label'],
      dtype='object')

In [67]:
from datetime import datetime 
start_time = datetime.now()
threshold = 0.08
training_result = {}

for shop_id in shop_ids:
    training_data = data[data.shop_id==shop_id].loc[:,map(lambda x:True if x != 'date' else False, data.columns.tolist())]
    x_train = training_data.iloc[:,:-1]
    y_train = training_data.iloc[:,-1]
    x_test = test_feature[test_feature.shop_id==shop_id].loc[:,map(lambda x:True if x != 'date' else False, test_feature.columns.tolist())]
    
    best_model = reporter.get_best_model(shop_id)
    if isinstance(best_model, basestring):
        best_model = tuple(best_model.split(','))
    
    for current_model in best_model:
        models[current_model].fit(x_train, y_train)
    
    y_pred = sum([models[current_model].predict(x_test) for model in best_model])/len(best_model)
    y_pred = map(lambda x:int(round(x,0)), y_pred)
    training_result[shop_id] = y_pred
#     for model_name, model in models.iteritems():
#         model.fit(x_train, y_train)
#         y_pred = model.predict(x_test)
#         training_result[model_name]['training_error'][shop_id] = sum(abs((y_pred - np.array(y_test))/(y_pred + np.array(y_test))))/len(y_pred)  
end_time = datetime.now()
print '耗时:{}'.format(end_time - start_time)

耗时:0:04:53.181596


In [68]:
training_result

{1: [351, 351, 372, 386, 351, 356, 355, 381, 381, 381, 381, 351, 351, 355],
 2: [31, 41, 43, 39, 112, 100, 101, 112, 102, 54, 60, 109, 45, 73],
 3: [55, 62, 52, 77, 119, 119, 64, 67, 66, 122, 125, 113, 112, 67],
 4: [87, 97, 84, 89, 180, 195, 96, 93, 104, 104, 125, 180, 191, 93],
 5: [146, 144, 143, 144, 146, 149, 150, 151, 154, 155, 157, 157, 158, 160],
 6: [81, 83, 84, 84, 85, 84, 83, 84, 84, 86, 87, 87, 88, 87],
 7: [134, 133, 124, 161, 205, 209, 124, 117, 111, 176, 153, 160, 181, 95],
 8: [79, 87, 82, 78, 77, 75, 75, 73, 74, 78, 72, 82, 85, 63],
 9: [184, 189, 188, 196, 247, 256, 218, 209, 210, 214, 207, 240, 250, 225],
 10: [54, 54, 50, 56, 58, 75, 88, 97, 97, 78, 83, 91, 87, 98],
 11: [88, 88, 89, 86, 69, 70, 88, 91, 91, 75, 72, 71, 73, 90],
 12: [68, 68, 69, 71, 73, 72, 70, 70, 70, 69, 69, 71, 70, 67],
 13: [242, 261, 261, 248, 208, 208, 242, 242, 248, 182, 191, 208, 208, 244],
 14: [64, 65, 65, 66, 65, 64, 68, 69, 69, 67, 67, 66, 65, 65],
 15: [95, 97, 95, 92, 135, 164, 105, 11

In [69]:
pickle.dump(training_result, open('prediction_1','w+'))

In [25]:
import pickle 
x = pickle.load(open('prediction_1', 'r+'))

In [19]:
result = []
for shop_id, pred in x.iteritems():
    if min(pred) < 0:
        temp = [shop_id] + map(lambda x:x+abs(min(pred))+10, pred)
    else:
        temp = [shop_id] + pred
    result.append(temp)

In [20]:
import pandas as pd
pd.DataFrame(result).to_csv('prediction_2017_02_28_raw.csv',header=False, index=False)

In [4]:
result = [[key]+value for key,value in x.iteritems()]

In [12]:
result

{1: [351, 351, 372, 386, 351, 356, 355, 381, 381, 381, 381, 351, 351, 355],
 2: [31, 41, 43, 39, 112, 100, 101, 112, 102, 54, 60, 109, 45, 73],
 3: [55, 62, 52, 77, 119, 119, 64, 67, 66, 122, 125, 113, 112, 67],
 4: [87, 97, 84, 89, 180, 195, 96, 93, 104, 104, 125, 180, 191, 93],
 5: [71, 39, 22, 39, 71, 119, 135, 151, 199, 215, 247, 247, 263, 296],
 6: [56, 71, 79, 79, 86, 79, 71, 79, 79, 94, 102, 102, 109, 102],
 7: [134, 133, 124, 161, 205, 209, 124, 117, 111, 176, 153, 160, 181, 95],
 8: [79, 87, 82, 78, 77, 75, 75, 73, 74, 78, 72, 82, 85, 63],
 9: [184, 189, 188, 196, 247, 256, 218, 209, 210, 214, 207, 240, 250, 225],
 10: [54, 54, 50, 56, 58, 75, 88, 97, 97, 78, 83, 91, 87, 98],
 11: [88, 88, 89, 86, 69, 70, 88, 91, 91, 75, 72, 71, 73, 90],
 12: [50, 50, 61, 83, 105, 94, 72, 72, 72, 61, 61, 83, 72, 39],
 13: [242, 261, 261, 248, 208, 208, 242, 242, 248, 182, 191, 208, 208, 244],
 14: [54, 60, 60, 66, 60, 54, 78, 84, 84, 72, 72, 66, 60, 60],
 15: [95, 97, 95, 92, 135, 164, 105, 11

In [13]:
filter(lambda a:min(a) < 0, [[key]+value for key,value in x.iteritems()])

[[1100, 2, 5, 7, 39, 110, 112, 24, -4, 1, 25, 60, 109, 106, 26],
 [1569, 71, 45, 73, 47, 42, 43, 77, 51, 44, -59, -20, 49, 50, 84]]

In [7]:
import pandas as pd
pd.DataFrame(result).to_csv('prediction_2017_02_28_raw.csv',header=False, index=False)

In [3]:
import pickle
from data_loader import DataLoader
from __init__ import project_path

data = pickle.load(open(project_path + '/data/user_pay_info', 'r+'))

In [10]:
import random
random.gauss(1,0.5)

1.1672820147479928

In [27]:
import numpy as np
import random


result = {}

for key, pred in x.iteritems():
    history = data[key].merchant_flow[-15:]
    if np.std(history)/np.std(pred) > 5:
        z = (pred - np.mean(pred))*np.std(history)/np.std(pred) + np.mean(pred)
        try:
            temp = map(lambda a:int(round(a)), z)
        except:
            temp = map(lambda a:int(round(a)), data[key].merchant_flow[-15:])
    else:
        temp = map(lambda a:int(round(a)), pred)
    if min(temp) < 0:
        temp = map(lambda a:abs(min(temp))+a+5, temp)
    result[key] = temp

/Users/myron/softwares/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: divide by zero encountered in double_scalars
/Users/myron/softwares/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in divide


In [29]:
len(result)

2000

In [30]:
import pandas as pd 
pd.DataFrame([[key]+value for key,value in result.iteritems()]).to_csv('prediction_2017_02_28_提高小波动率商家.csv',header=False, index=False)

In [57]:
map(lambda x:int(round(x,0)), training_result[1].tolist())

[303, 349, 380, 387, 380, 359, 349, 350, 370, 359, 385, 359, 359, 349]

In [ ]:
# from datetime import datetime 
# start_time = datetime.now()
# threshold = 0.08
# training_errors = {}

# for shop_id in [1]:
#     training_data = data[data.shop_id==shop_id].loc[:,map(lambda x:True if x != 'date' else False, data.columns.tolist())]
#     test_data = data[data.shop_id==shop_id].loc[:,map(lambda x:True if x != 'date' else False, data.columns.tolist())]
#     x_train = training_data.iloc[:,:-1]
#     y_train = training_data.iloc[:,-1]
#     x_test = test_data.iloc[:,:-1]
#     y_test = test_data.iloc[:,-1]
    
#     best_model = reporter.get_best_model(shop_id)
#     if isinstance(best_model, basestring):
#         best_model = tuple(best_model.split(','))
    
#     for current_model in best_model:
#         models[current_model].fit(x_train, y_train)
    
#     y_pred = sum([models[current_model].predict(x_test) for model in best_model])/len(best_model)
#     training_error = sum(abs((y_pred - np.array(y_test))/(y_pred + np.array(y_test))))/len(y_pred) 
#     training_errors[shop_id] = training_error
    
# #     for model_name, model in models.iteritems():
# #         model.fit(x_train, y_train)
# #         y_pred = model.predict(x_test)
# #         training_result[model_name]['training_error'][shop_id] = sum(abs((y_pred - np.array(y_test))/(y_pred + np.array(y_test))))/len(y_pred)  
# end_time = datetime.now()
# print '耗时:{}'.format(end_time - start_time)
# training_errors